In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import time
import re
import nltk
import math
import os
import matplotlib.pyplot as plt
import sklearn.metrics
import json 
from datetime import datetime

from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import Perceptron

from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score

import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pickle

%matplotlib inline


In [2]:
# load csv file into Pandas dataframe table format
df_users = pd.read_csv('takehome_users.csv')

df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
# load csv file into Pandas dataframe table format
df_engagement = pd.read_csv('takehome_user_engagement.csv')

df_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
df_users.dtypes

object_id                       int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
dtype: object

In [5]:
df_users.describe

<bound method DataFrame.describe of        object_id        creation_time                 name  \
0              1  2014-04-22 03:53:30       Clausen August   
1              2  2013-11-15 03:45:04        Poole Matthew   
2              3  2013-03-19 23:14:52    Bottrill Mitchell   
3              4  2013-05-21 08:09:28      Clausen Nicklas   
4              5  2013-01-17 10:14:20            Raw Grace   
5              6  2013-12-17 03:37:06        Cunha Eduardo   
6              7  2012-12-16 13:24:32         Sewell Tyler   
7              8  2013-07-31 05:34:02    Hamilton Danielle   
8              9  2013-11-05 04:04:24           Amsel Paul   
9             10  2013-01-16 22:08:03         Santos Carla   
10            11  2013-12-26 03:55:54       Paulsen Malthe   
11            12  2014-04-17 23:48:38      Mathiesen L�rke   
12            13  2014-03-30 16:19:38        Fry Alexander   
13            14  2012-10-11 16:14:33          Rivera Bret   
14            15  2013-07-16 21:33

In [6]:
df_engagement.dtypes

time_stamp    object
user_id        int64
visited        int64
dtype: object

In [7]:
df_engagement.shape

(207917, 3)

In [8]:
#df_engagement.describe

In [9]:
df_engagement.time_stamp = df_engagement.time_stamp.apply(pd.to_datetime)

In [10]:
df_engagement.dtypes

time_stamp    datetime64[ns]
user_id                int64
visited                int64
dtype: object

In [11]:
df_engagement.head(5)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [12]:
df_engagement['Date'] = df_engagement.time_stamp.dt.date

In [13]:
df_engagement.dtypes

time_stamp    datetime64[ns]
user_id                int64
visited                int64
Date                  object
dtype: object

In [14]:
df_engagement.head()

,time_stamp,user_id,visited,Date
0,2014-04-22 03:53:30,1,1,2014-04-22
1,2013-11-15 03:45:04,2,1,2013-11-15
2,2013-11-29 03:45:04,2,1,2013-11-29
3,2013-12-09 03:45:04,2,1,2013-12-09
4,2013-12-25 03:45:04,2,1,2013-12-25


In [15]:
# Create truncated dataframe to check if functions work or not
df_trunc = df_engagement.iloc[300:755,:]

df_trunc.head()

,time_stamp,user_id,visited,Date
300,2014-05-31 22:08:03,10,1,2014-05-31
301,2014-06-01 22:08:03,10,1,2014-06-01
302,2014-06-02 22:08:03,10,1,2014-06-02
303,2014-06-03 22:08:03,10,1,2014-06-03
304,2013-12-27 03:55:54,11,1,2013-12-27


In [16]:
#def simple_check(df,col):
    #adpt_stats = []
    #usr_list = []
    #hug = 0
    #for name, group in df.groupby([col]):
        #usr_list.append(name)
        #print('User:'+ str(name))
        #print('Number of total visits:'+ str(len(group)))
        #dates = list(group.Date)
        #print(len(group.Date))
        #print(type(dates))
        #for date in dates:
            #print(dates.index(date))
            #if (dates.index(date) == len(dates)-1):
                #print('This is the last recorded visit for this user')
                #print(date)
                
            #else:
                #print(date)
            
        #print

In [17]:
#simple_check(df_trunc,'user_id')

In [18]:
def simple_check_2(df,col):
    adpt_stats = []
    usr_list = []
    hug = 0
    # iterate over each group, in this case each group corresponds to a user
    for name, group in df.groupby([col]):
        usr_list.append(name)
        #print('User:'+ str(name))
        #print('Number of total visits:'+ str(len(group)))
        # create a python list of datetime objects called dates
        dates = list(group.Date)
        if (len(dates) <=2):
            adpt_stats.append(0)
            #print('NOT AN ADOPTER')
        else:
            #print(len(dates))
            for date in group.Date:
                status = [x for x in dates if (x - date <= pd.Timedelta('7 days'))\
                            & (x - date >= pd.Timedelta('0 days'))]
                #print(date)
                #print(status)
                # if the number of visits in any 7-day period is 3 or more
                # update the adoption status as a 1 and break out of loop
                if (len(status) >2):
                    #print('Adopter')
                    adpt_stats.append(1)
                    break
                # check if this is the last date on the list & the number
                # of visits in any 7-day period is < 3, if so update adopter status as 0
                if (len(status) < 2) & ((dates.index(date) == len(dates)-1)):
                    adpt_stats.append(0)
                    #print('This user had a total or more than 2 visits')
                    #print('BUT did NOT visit at least 3 times in ANY seven day period')
        #print
        
    #print(len(usr_list))
    #print(len(adpt_stats))
    #print('*** That is All***')
    
    return usr_list, adpt_stats

In [19]:
usr_list, adopt_list = simple_check_2(df_trunc,'user_id')

In [20]:
print(len(usr_list))
print
print(len(adopt_list))

41

41


In [21]:
print(usr_list)
print
print(adopt_list)

[10, 11, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 33, 36, 37, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63]

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1]


In [22]:
usr_list, adopt_list = simple_check_2(df_engagement,'user_id')

In [23]:
print(len(usr_list))
print
print(len(adopt_list))

8823

8823


In [24]:
# Create pandas dataframe using dictionary of user data
status_dict = {'user_id': usr_list, 'adopt_status': adopt_list}

df_3 = pd.DataFrame(status_dict)

df_3.head()

,adopt_status,user_id
0,0,1
1,1,2
2,0,3
3,0,4
4,0,5


In [25]:
df_3.tail()

,adopt_status,user_id
8818,0,11996
8819,0,11997
8820,0,11998
8821,0,11999
8822,0,12000


In [26]:
df_3.adopt_status.value_counts()

0    7167
1    1656
Name: adopt_status, dtype: int64

## Merge Dataframes

In [27]:
df_users.columns

Index([u'object_id', u'creation_time', u'name', u'email', u'creation_source',
       u'last_session_creation_time', u'opted_in_to_mailing_list',
       u'enabled_for_marketing_drip', u'org_id', u'invited_by_user_id'],
      dtype='object')

In [28]:
# rename column of df_users

df_users.rename(columns={"object_id": "user_id"},inplace = True)

df_users.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [29]:
df_users.shape

(12000, 10)

In [30]:
df_user_data = pd.merge(left=df_users, right=df_3, left_on='user_id', right_on='user_id')

df_user_data.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopt_status
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0


In [31]:
df_user_data.shape

(8823, 11)

In [32]:
df_user_data.columns

Index([u'user_id', u'creation_time', u'name', u'email', u'creation_source',
       u'last_session_creation_time', u'opted_in_to_mailing_list',
       u'enabled_for_marketing_drip', u'org_id', u'invited_by_user_id',
       u'adopt_status'],
      dtype='object')

In [33]:
df_user_data.dtypes

user_id                         int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
adopt_status                    int64
dtype: object

In [34]:
df_user_data[['user_id','name','email','creation_source','adopt_status']].head(15)

,user_id,name,email,creation_source,adopt_status
0,1,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,0
1,2,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1
2,3,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,0
3,4,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,0
4,5,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,0
5,6,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,0
6,7,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,0
7,10,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1
8,11,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,0
9,13,Fry Alexander,AlexanderDFry@cuvox.de,ORG_INVITE,0


In [35]:
# save the dataframe as csv file for future use
df_user_data.to_csv('relax_users_data.csv',index = None)

user = pd.read_csv('relax_users_data.csv')

user.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopt_status
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0


## Convert Categorical Features

In [36]:
# https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02
from sklearn.preprocessing import LabelEncoder

cle = LabelEncoder()
df_user_data['creation_source_label'] = cle.fit_transform(df_user_data['creation_source'])

# convert string into date time object
df_user_data.creation_time = df_user_data.creation_time.apply(pd.to_datetime)

df_user_data.dtypes

user_id                                int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                       object
last_session_creation_time           float64
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                   float64
adopt_status                           int64
creation_source_label                  int64
dtype: object

In [37]:
df_user_data.columns

Index([u'user_id', u'creation_time', u'name', u'email', u'creation_source',
       u'last_session_creation_time', u'opted_in_to_mailing_list',
       u'enabled_for_marketing_drip', u'org_id', u'invited_by_user_id',
       u'adopt_status', u'creation_source_label'],
      dtype='object')

In [38]:
df_user_data.describe()

,user_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopt_status,creation_source_label
count,8823.000000,8.823000e+03,8823.000000,8823.000000,8823.000000,4776.000000,8823.000000,8823.000000
mean,6019.821716,1.379279e+09,0.252295,0.151989,142.572254,5980.495394,0.187691,1.807775
std,3464.251001,1.953116e+07,0.434354,0.359031,124.176422,3394.211361,0.390488,1.373882
min,1.000000,1.338452e+09,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
25%,3017.500000,1.363195e+09,0.000000,0.000000,30.000000,3071.000000,0.000000,1.000000
50%,6034.000000,1.382888e+09,0.000000,0.000000,109.000000,5947.000000,0.000000,1.000000
75%,9029.500000,1.398443e+09,1.000000,0.000000,239.000000,8857.250000,0.000000,3.000000
max,12000.000000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000,1.000000,4.000000


In [39]:
df_user_data.isna().sum()

user_id                          0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time       0
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            4047
adopt_status                     0
creation_source_label            0
dtype: int64